# MODELO



### PREPARACIÓN DE LA DATA

El proceso de preparación de la data para el modelo de entrenamiento consiste en obtener listas de vectores, cada vector contiene la sentencia de pregunta y la sentencia de respuesta. En este proceso también hacemos un mapeo de cada palabra a índices y de índice palabras, esto se almacenará en diccionarios, además de llevar el conteo de cuántas veces una palabra se repite. Esto nos permitirá obtener el índice que le corresponde a cada palabra y viceversa.

In [1]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/MyDrive/iA

Mounted at /gdrive
/gdrive/.shortcut-targets-by-id/1ccQ0NRVtxcnMDQHOrldbGFQDfXPWk9Jj/iA


In [2]:
%ls

20.txt
BackwardForward.ipynb
checkpoint-10ML-30ep-META-512bz-loss29-valoss41.pt
checkpoint-10ML-30ep-META-512bz-loss30-valoss41-con-tildes.pt
checkpoint-10ML-60ep-META-512bz-loss26-valoss46-con-tildes2.pt
checkpoint-15ML-20EP.pt
checkpoint-15ML-30ep-META-512bz-loss40-valoss54-con-tildes.pt
checkpoint-20ML-15ep-data-METALW-512bz-con-tildes-300em-300h.pt
checkpoint-20ML-25ep-data-METALW-512bz-con-tildes-300em-300h.pt
checkpoint-25ML-13EP-300BS.pt
checkpoint-25ML-15ep-data-METALW-512bz-con-tildes-200em-200h.pt
checkpoint-25ML-25ep-data-METALW-512bz-con-tildes-200em-200h.pt
checkpoint2.pt
checkpoint-50ML-20EP.pt
checkpoint.pt
continnuacion/
cultura_general_1
data_freider.txt
Data_PC2/
final.txt
GRL_Book.pdf
METALW.txt


In [3]:
import torch
from torch.jit import script
import torch.nn as nn
import random
import re
import unicodedata
from io import open
import itertools
from tqdm import tqdm

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

MAX_LENGTH = 10 # Cantidad de palabras máxima por cada sentencia

In [4]:
# Definiciones:
#   Linea: Pares de sentencias separadas por un padding \t
#   Sentencia: Texto de la pregunta o la respuesta.
#   Par: Un vector, cada vector contiene dos senticias: pregunta y la respuesta

PAD_token = 0  # Token para rellenar las sentencias con una cantidad menor a MAX_LENGTH
SOS_token = 1  # Token que indica el inicio de la sentencia
EOS_token = 2  # Token que indica el final de la sentencia

# Objeto Voc: 
#   Procesará cada sentencia de cada línea. 
#   Nos ayudará a generar una mapeo de cada palabra a indices (números)
#   lo que permitirá obtener el índice que corresponde a cada palabra, la palabra que le
#   corresponde a cada índice y la cantidad de veces que una palabra se repíte.
class Voc:
    def __init__(self, name):
        self.name = name
        self.trimmed = False
        self.word2index = {"PAD":PAD_token , "SOS":SOS_token , "EOS":EOS_token }
        self.word2count = {}
        self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
        self.num_words = 3  # Los 3 tokens inicializados SOS, EOS, PAD

    def agregarSentencia(self, sentence):
        for word in sentence.split(' '):
            self.agregarPalabra(word)

    def agregarPalabra(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.num_words
            self.word2count[word] = 1
            self.index2word[self.num_words] = word
            self.num_words += 1
        else:
            self.word2count[word] += 1

    def indiceDeSentencia(self, sentencia):
        return [self.word2index[word] for word in sentencia.split(' ')] + [EOS_token]

    def sentenciaDeIndice(self, indice):
        return [self.index2word[idx] for idx in indice]

    # Remueve las palabras que se repiten menos de una cierta cantidad de veces
    def trim(self, min_count):
        if self.trimmed:
            return
        self.trimmed = True

        keep_words = []

        for k, v in self.word2count.items():
            if v >= min_count:
                keep_words.append(k)

        print('keep_words {} / {} = {:.4f}'.format(
            len(keep_words), len(self.word2index), len(keep_words) / len(self.word2index)
        ))

        # Reinicializamos los diccionarios
        self.word2index = {"PAD":PAD_token , "SOS":SOS_token , "EOS":EOS_token }
        self.word2count = {}
        self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
        self.num_words = 3 # Los 3 tokens inicializados SOS, EOS, PAD

        for word in keep_words:
            self.agregarPalabra(word)

In [6]:
# Función que normalizará cada sentencia
def unicodeToAscii(s):
    return ''.join(
        # c for c in unicodedata.normalize('NFD', s) # Normalizará y eliminará las tildes y la ñ
        c for c in unicodedata.normalize('NFC', s) # Normalizará y mantiene las tildes y la ñ
        if unicodedata.category(c) != 'Mn'
    )

# Normalizamos cada sentencia
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip()) # Normalizamos y pasamos todas las letras a minúsculas
    # Pasamos cada sentencia a minúsculas, removemos los espacios y carácteres que no son letras excluyendo los números
    s = re.sub(r"([.¡!¿?])", r" \1", s)  # Mantenemos los signos interrogación y exclamación de apertura y cierre
    s = re.sub(r"[^A-zÁ-ú.¡!¿?0-9]+", r" ", s) # Mantenemos las tildes y números
    # s = re.sub(r"\¿", r"¿ ", s) # Agregamos un espacio a los signos de interrogación para que se cuente como una palabra
    # s = re.sub(r"\?", r" ?", s) # Agregamos un espacio a los signos de interrogación para que se cuente como una palabra
    # s = re.sub(r"\¡", r"¡ ", s) # Agregamos un espacio a los signos de exclamación para que se cuente como una palabra
    # s = re.sub(r"\!", r" !", s) # Agregamos un espacio a los signos de exclamación para que se cuente como una palabra
    # s = re.sub(r"\s+", r" ", s).strip() # Eliminamos los espacios demás
    # s = re.sub(r"\.", r"", s).strip() # Eliminamos los puntos

    # s = re.sub(r"[^A-z.¡!¿?0-9]+", r" ", s) # Elimina tildes
    s = re.sub(r"\.", r"", s)
    s = re.sub(r"\¿\s+", r"¿", s) # Elimina espacios alrededor del signo de interrogación
    s = re.sub(r"\s+\?", r"?", s) # Elimina espacios alrededor del signo de interrogación
    s = re.sub(r"\¡\s", r"¡", s) # Elimina espacios alrededor del signo de exclamación
    s = re.sub(r"\!\s", r"!", s) # Elimina espacios alrededor del signo de exclamación
    s = re.sub(r"\s+", r" ", s).strip() # Elimina los espacios demás
    return s



# Leemos las lineas del archivo y devolvemos los pares y un objeto Voc
def readVocs(datafile, corpus_name):
    print("Leyendo líneas...")
    # Leemos el archivo y devuelve una lista de líneas
    lines = open(datafile, encoding='utf-8').\
        read().strip().split('\n')
    # Dividimos cada linea en pares, normaliza normalizando cada sentencia
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]
    voc = Voc(corpus_name)
    # Devuelve el objeto vocabulario y los pares
    return voc, pairs

# Retorna True si ambas sentencias en el par tienen una cantidad de palabras menores que MAX_LENGTH
def filtrarPar(p):
    # Las sentencias de entrada, necesitamos un espacio para el token SOS
    return len(p[0].split(' ')) < MAX_LENGTH and len(p[1].split(' ')) < MAX_LENGTH

# Filtra los pares usando la función filtrarPar
def filtrarPares(pairs):
    return [pair for pair in pairs if filtrarPar(pair)]

# Usando las funciones definidas arriba generamos el diccionario que mapea de palabras a índices
# devolverá el objeto voc y la lista de pares
def loadPrepareData(corpus, corpus_name, datafile, save_dir):
    print("Empieza la preparación de la data ...")
    voc, pairs = readVocs(datafile, corpus_name)
    print("Se leyó {!s} pares de sentencias".format(len(pairs)))
    pairs = filtrarPares(pairs)
    print("Filtrado {!s} pares de sentencias".format(len(pairs)))
    print("Contando las palabras...")

    for pair in pairs:
        # Agregamos cada sentencia al objeto Voc para hacer el mapeo
        voc.agregarSentencia(pair[0])
        voc.agregarSentencia(pair[1])
        
    print("Cantidad total de palabras:", voc.num_words)
    
    
    return voc, pairs



save_dir = "./"
datafile = "METALW.txt"
corpus = "./"
corpus_name = "dataf_s2s"
voc, pairs = loadPrepareData(corpus, corpus_name, datafile, save_dir)

# Imprimimos una muestra de la data para verificar su estructura
print("\npares:")

for pair in pairs[:10]:
    print(pair)

# Fuente: https://pytorch.org/tutorials/beginner/chatbot_tutorial.html?highlight=chatbot%20tutorial

Empieza la preparación de la data ...
Leyendo líneas...
Se leyó 319593 pares de sentencias
Filtrado 175166 pares de sentencias
Contando las palabras...
Cantidad total de palabras: 40305

pares:
['hola ¿en qué puedo ayudarle?', 'sí']
['sí', '¿cómo puedo ser de ayuda?']
['¿cómo puedo ser de ayuda?', 'quiero saber sobre la política que tengo']
['quiero saber sobre la política que tengo', 'bien ¿puedo conseguir tu nombre por favor?']
['bien ¿puedo conseguir tu nombre por favor?', '¿cubre los daños causados por el agua?']
['hola ¿en qué puedo ayudarle?', 'tengo una pregunta sobre mi política']
['tengo una pregunta sobre mi política', 'seguro ¿puedes decirme tu número de póliza?']
['seguro ¿puedes decirme tu número de póliza?', '3425512']
['3425512', 'bien tengo tu póliza aquí ¿cuál es tu pregunta?']
['bien tengo tu póliza aquí ¿cuál es tu pregunta?', 'cubre los daños causados por el agua?']
